# III. Preparación de datos

En esta sección se entrenan **distintos modelos** sobre los datos limpios con el objetivo de **predecir el número de ventas para el mes de noviembre de 2015 por tienda y artículo**. Se prueban los modelos de **regresión con regularización ridge, regularización lasso y XGBoost**.

Es importante desctacar que en la sección anterior de Procesamiento de Datos, se logró obtener la base de datos limpia que sirve para entrenar modelos utilizando R. Sin embargo, el data frame con las **10,913,850 observaciones y 44 columnas** que se obtuvieron como resultado y la manera en que R guarda y escribe CSVs, se generarón archivos muy pesados. En consecuencia, leer la base limpia en Python y Jupyter Notebook ocasionaba problemas de memoría, matando al Kernel.

Tomando en cuenta lo anterior y sin olvidar la instrucción inicial de limpiar los datos en R, en este documento de modelado se replica el código de limpieza que se obtuvó y se describió en la sección de Procesamiento de Datos. Esta implementación generar el mismo dataframe, pero optimizado y con menor peso que funciona bien en Python para entrenar modelos.

## Cargamos las librerías

Las librerías que se utilizan para el modelado son las siguientes.

In [21]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import TimeSeriesSplit
import warnings; warnings.simplefilter('ignore')

## Cargamos los datos crudos

In [22]:
DATA_FOLDER = 'datos'
transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train.csv'))
items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'), encoding = "Windows-1251")
shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
test            = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))

## Generamos los datos limpios

**Problemas:**

* No todos los artículos se venden en las mismas tiendas, todos los meses
* Matriz rala

**Solución:**
    
* Completar con un grid
* Imputar ceros

Un problema es que hay duplas (tienda, artículo) para las cuales no hay registro en algunos meses. Es decir, no en todos los meses del año se vendieron todos los artículos en todas las tiendas. Por lo tanto, no se tienen las series de tiempo completas. Para resolver este problema, vamos a completar dichas series de tal manera que el algoritmo sea capaz de aprender las irregularidades en las ventas de 1C.

Generamos el grid con todas las combinaciones posibles de mes, tienda y artículo que dió como resultado un dataset de 10,913,850 de filas.

In [23]:
# Hacemos un merge de las los itemas a las transacciones por item id
transactions = pd.merge(transactions, items, on='item_id', how='left')
transactions = transactions.drop('item_name', axis=1)
transactions.head()

# Construimos el grid 
from itertools import product
index_cols = ['shop_id', 'item_id', 'date_block_num']
# Inicializas array
grid = []
# Para cada mes sacar tiendas e items únicos, calculas el plano cartesiano y lo guardas
for block_num in transactions['date_block_num'].unique():
    cur_shops = transactions.loc[transactions['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = transactions.loc[transactions['date_block_num'] == block_num, 'item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])), dtype='int32'))
# Apilas cada uno de los planos cartesianos para tener el grid    
grid = pd.DataFrame(np.vstack(grid), columns = index_cols, dtype=np.int32)

Creamos las características de ventas promedio por mes, tienda y producto con las cuáles se crean las características que necesitamos de manera rezagada.

In [24]:
# Cálculamos medias por mes, tienda y producto.
mean_transactions = transactions.groupby(['date_block_num', 'shop_id', 'item_id']).agg(
    {'item_cnt_day':'sum','item_price':np.mean}).reset_index()
# Pegamos estos promedios al grid
mean_transactions = pd.merge(grid,mean_transactions,on=['date_block_num', 'shop_id', 'item_id'],how='left')

# Se juntan estas estadísticas al grid
mean_transactions = pd.merge(grid,mean_transactions,on=['date_block_num', 'shop_id', 'item_id'],how='left')

Unicamente tenemos missings en item_price_mean y item_cnt_month. Los missings en item_cnt_month corresponden a que no hubo ventas para ese artículo, en ese mes, en esa tienda; porlo tanto, sustituimos estos NA's con 0.


In [25]:
# Imputamos con ceros
mean_transactions = pd.merge(grid,mean_transactions,on=['date_block_num', 'shop_id', 'item_id'],how='left').fillna(0)
# Se juntan los datos de categorías de artículo
mean_transactions = pd.merge(mean_transactions, items, on='item_id',how='left')

## Ingeniería de características

Se generan **9 nuevas variables** que serán utilizadas posteriormente para obtener los reszagos de interes.

**Por artículo de interés en todas las tiendas**:

* precio promedio
* ventas totales
* ventas promedio

**Por tienda de interés**:

* precio promedio del producto
* ventas totales del producto
* ventas promedio del producto

**Por categoría del producto de interés**: 

* precio promedio
* ventas totales
* ventas promedio


En total se crean 36 nuevas variables. De tal manera que nuestra **matriz de diseño queda de 10,849,431 filas y 44 columnas**. 

<img src="img/feature_construction.png">


Con los datos originales creamos las siguientes características:

In [26]:
# El proceso se repite por type_id
for type_id in ['item_id', 'shop_id', 'item_category_id']:
    # Se van a calcular usando una tupla el precio promedio, las ventas totales, las ventas diarias
    for column_id, aggregator, aggtype in [('item_price',np.mean,'avg'),('item_cnt_day',np.sum,'sum'),('item_cnt_day',np.mean,'avg')]:
        # Agregas el grid con las medias por la categoría de agregación type_id
        mean_df = transactions.groupby([type_id,'date_block_num']).aggregate(aggregator).reset_index()[[column_id,type_id,'date_block_num']]
        # A estas nuevas variables, les asignas su nombre por categoría
        mean_df.columns = [type_id+'_'+aggtype+'_'+column_id,type_id,'date_block_num']
        # Agregas características al grid
        mean_transactions = pd.merge(mean_transactions, mean_df, on=['date_block_num',type_id], how='left')

Las variables que se crearon se agregaron al dataset completo con un left join. 


Dado que en la verdadera tarea de predicción no se van a tener los datos actuales, sino los datos de los meses previos, creamos rezagos mensuales, bimestrales, trimestrales y semestrales de las variables de precio y cantidad creadas en el paso anterior.

In [27]:
# Se crean las características rezagadas
lag_variables  = list(mean_transactions.columns[7:])+['item_cnt_day']
# Creamos 4 lags
lags = [1, 2, 3, 6]
from tqdm import tqdm_notebook
# Loop por lag
for lag in tqdm_notebook(lags):
    # Creas una copia del grid populado
    sales_new_df = mean_transactions.copy()
    # Incrementas la indicadora del mes por el lag
    sales_new_df.date_block_num += lag
    # Extraes las variables categóricas de identificación más los lags
    sales_new_df = sales_new_df[['date_block_num','shop_id','item_id']+lag_variables]
    # Cambias los nombres para que indiquen a que lag pertenecen
    sales_new_df.columns = ['date_block_num','shop_id','item_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variables]
    # Haces un merge al grid populado
    mean_transactions = pd.merge(mean_transactions, sales_new_df,on=['date_block_num','shop_id','item_id'] ,how='left')

Al generar un rezago de tamaño $n$ se obtienene $n$ valores faltantes. Imputamos los NA's de los rezagos (primeros $n$ valores para rezago de tamaño $n$ en cada variable) con el primer valor observado (correspondiente a la observación $n+1$).


In [28]:
#  Imputación de NAs después de rezagar
mean_transactions = mean_transactions[mean_transactions['date_block_num'] > 12]
# Rellenamos con ceros item_cnt
# Rellenamos con medianas los precios
for feat in mean_transactions.columns:
    if 'item_cnt' in feat:
        mean_transactions[feat]=mean_transactions[feat].fillna(0)
    elif 'item_price' in feat:
        mean_transactions[feat]=mean_transactions[feat].fillna(mean_transactions[feat].median())

Ahora se quitan todos los datos que no tendremos al momento de hacer la predicción, es decir todos los datos sin rezago. Solo se conserva "item_cnt_day"

In [29]:
cols_to_drop = lag_variables[:-1] + ['item_price', 'item_name']
training = mean_transactions.drop(cols_to_drop,axis=1)

Las columnas del dataframe de entrenamiento limpio y con características nuevas se muestra a continuación.

*Información del producto, tienda y mes de interés*

1. **shop_id**: Id de la tienda
2. **item_id**: Id del producto
3. **date_block_num**: Id del mes, en este caso 33 para el mes de octubre de 2015
4. **item_cnt_day**: Ventas promedio mensuales para el artículo
5. **item_category_id**: Id de la categoría del producto

*Rezago mensual: datos agrupados por producto, tienda y categoría*

6. **item_id_avg_item_price_lag_1**: artículo de interés en todas las tiendas, precio promedio del mes anterior
7. **item_id_sum_item_cnt_day_lag_1**: artículo de interés en todas las tiendas, ventas totales del mes anterior
8. **item_id_avg_item_cnt_day_lag_1**: artículo de interés en todas las tiendas, ventas promedio del mes anterior
9. **shop_id_avg_item_price_lag_1**: en la tienda de interés, precio promedio del producto en el mes anterior.
10. **shop_id_sum_item_cnt_day_lag_1**: en la tienda de interés, ventas totales del producto en el mes anterior.
11. **shop_id_avg_item_cnt_day_lag_1**: en la tienda de interés, ventas promedio del producto en el mes anterior.
12. **item_category_id_avg_item_price_lag_1**: en la categoría del producto de interés, precio promedio en el mes anterior.
13. **item_category_id_sum_item_cnt_day_lag_1**: en la categoría del producto de interés, ventas totales en mes anterior.
14. **item_category_id_avg_item_cnt_day_lag_1**: en la categoría del producto de interés, ventas promedio del mes anterior.
15. **item_cnt_day_lag_1**: artículo y tienda de interés, ventas en el mes anterior en la tienda de in 

*Rezago bimestral: datos agrupados por producto, tienda y categoría*

16. **item_id_avg_item_price_lag_2**: ídem, dos meses atrás
17. **item_id_sum_item_cnt_day_lag_2**: ídem, dos meses atrás
18. **item_id_avg_item_cnt_day_lag_2**: ídem, dos meses atrás
19. **shop_id_avg_item_price_lag_2**: ídem, dos meses atrás
20. **shop_id_sum_item_cnt_day_lag_2**: ídem, dos meses atrás
21. **shop_id_avg_item_cnt_day_lag_2**: ídem, dos meses atrás
22. **item_category_id_avg_item_price_lag_2**: ídem, dos meses atrás
23. **item_category_id_sum_item_cnt_day_lag_2**: ídem, dos meses atrás
24. **item_category_id_avg_item_cnt_day_lag_2**: ídem, dos meses atrás
25. **item_cnt_day_lag_2**:

*Rezago cuatrimestral: datos agrupados por producto, tienda y categoría*

26. **item_id_avg_item_price_lag_3**: ídem, tres meses atrás
27. **item_id_sum_item_cnt_day_lag_3**: ídem, tres meses atrás
28. **item_id_avg_item_cnt_day_lag_3**: ídem, tres meses atrás
29. **shop_id_avg_item_price_lag_3**: ídem, tres meses atrás
30. **shop_id_sum_item_cnt_day_lag_3**: ídem, tres meses atrás
31. **shop_id_avg_item_cnt_day_lag_3**: ídem, tres meses atrás
32. **item_category_id_avg_item_price_lag_3**: ídem, tres meses atrás
33. **item_category_id_sum_item_cnt_day_lag_3**: ídem, tres meses atrás
34. **item_category_id_avg_item_cnt_day_lag_3**: ídem, tres meses atrás
35. item_cnt_day_lag_3**: ídem, tres meses atrás

*Rezago semestral: datos agrupados por producto, tienda y categoría*

36. **item_id_avg_item_price_lag_6**: ídem, seis meses atrás
37. **item_id_sum_item_cnt_day_lag_6**: ídem, seis meses atrás
38. **item_id_avg_item_cnt_day_lag_6**: ídem, seis meses atrás
39. **shop_id_avg_item_price_lag_6**: ídem, seis meses atrás
40. **shop_id_sum_item_cnt_day_lag_6**: ídem, seis meses atrás
41. **shop_id_avg_item_cnt_day_lag_6**: ídem, seis meses atrás
42. **item_category_id_avg_item_price_lag_6**: ídem, seis meses atrás
43. **item_category_id_sum_item_cnt_day_lag_6**: ídem, seis meses atrás
44. **item_category_id_avg_item_cnt_day_lag_6**: ídem, seis meses atrás
45. **item_cnt_day_lag_6**: ídem, tres meses atrás

In [175]:
# Guardamos una copia el directorio de modelos
training.to_csv("Data_Modelos/Entrenamiento_Modelos2.csv", index = False, header=True)

# IV. Modelado

## Entrenamiento de modelos

A partir de esta parte del documento se procede a entrenar los modelos. Primero se procesan los datos en el formato que requieren las librerías de los modelos, se estandarizan las variables explicativas y se realiza una validación cruzada para encontrar el mejor modelo.

Se ajustará primero una regresión ridge, luego una lasso y finalmente un xgboost.

In [2]:
# Cargamos la base de datos de entrenamiento limpia y con rezagos.
#training = pd.read_csv("Data_Modelos/Entrenamiento_Modelos2.csv", header=1)

### Preprocesamiento de datos

Separamos el conjunto de entrenamiento en la matriz de variables explicativas y el vector de variables respuesta.

In [31]:
# Separamos el conjunto de entrenamiento en y y X 
X = training.iloc[:, training.columns != 'item_cnt_day'].values 
y = training.iloc[:, training.columns == 'item_cnt_day'].values

### Estandarizamos datos

Se aplica un procedimiento de estandarización de las variables explicativas. Esto con el objeto de ayudar a encontrar un valor óptimo de los parámetros de manera eficiente y también porque así lo requieren los modelos de regularización Ridge y Lasso. 

En este proyecto se utiliza el método restar la media y dividir por la norma l2.

In [103]:
# Escalamiento en la matriz de diseño.
#scaler = MinMaxScaler()
#X_scaled = scaler.fit_transform(X)

### Construcción de conjuntos de valiadción

Para evitar el sobre ajuste de los modelos y poder comparar su efectividad para predecir observaciones no antes vistas, el proceso de validación cruzada que se lleva acabo para un problema de series de tiempo.

Se parte el conjunto de entrenamiento en **5 splits de diferente longitud**, como se muestra en la imagen. Cada split se va incrementando en el tiempo para reflejar el efecto temporal de las series. **Cada split tiene su conjunto de entrenamiento y validación**. 

Al final el error de prueba se calcula promediando el error de prueba de cada split.

<img src = "Data_Modelos/crossvalidation.png">

In [4]:
# Creamos conjuntos de validación cruzada
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)
my_cv = TimeSeriesSplit(n_splits=5).split(X)

###  Modelo de Regresión Ridge

Se ajusta un modelo de regresión lineal con regularización ridge, para penalizar los modelos con parámetros muy grandes. 

* Se prueba distintos parámetros de **alpha 0.1, 1, 10, 20 y 30** donde a más grande el alpha más se castiga aquellos parámetros con valores altos. Entre más cercano el parámetro de alpha a cero, más parecido el resultado es a la regresión lineal.

* Se utilizó la función de pérdida "negative mean squared error".

In [125]:
# Modelos con Validación Cruzada con Grid Search
## Ridge
ridge = Ridge(normalize=True)
## 5 splits
my_cv = TimeSeriesSplit(n_splits=5).split(X)
## regularización
param_grid = {'alpha': [0.1, 1, 10, 20, 30]}
## grid search

ridge_cv = GridSearchCV(estimator = ridge,
                        param_grid = param_grid,
                        scoring = 'neg_mean_squared_error',
                        cv = my_cv
                        )
ridge_cv.fit(X,y)

GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x000002430C4C7780>,
       error_score='raise',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.1, 1, 10, 20, 30]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

Los resultados de la validación son los siguientes:

In [127]:
pd.DataFrame(ridge_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,8.563997,2.012142,0.120198,0.017758,0.1,{'alpha': 0.1},-2.347157,-6.743744,-9.106981,-9.368888,...,-8.531836,4.135702,1,-3.744499,-2.993961,-4.165695,-5.375480,-6.160724,-4.488072,1.137933
1,8.621394,2.088545,0.113001,0.013040,1,{'alpha': 1},-2.541313,-7.648511,-10.015086,-8.904451,...,-8.951469,4.213421,2,-4.212809,-3.366871,-4.618107,-5.920064,-6.641781,-4.951926,1.180087
2,8.886791,2.081479,0.112000,0.011333,10,{'alpha': 10},-4.513691,-10.705913,-14.344333,-9.326813,...,-11.275818,4.424047,3,-6.861300,-5.693343,-7.184180,-8.916298,-9.240763,-7.579177,1.324877
3,8.790578,2.024215,0.112600,0.014854,20,{'alpha': 20},-5.357072,-11.748445,-15.907999,-9.848393,...,-12.192559,4.497311,4,-7.976637,-6.678059,-8.240841,-10.120078,-10.251115,-8.653346,1.358972
4,8.762989,2.026495,0.111600,0.011842,30,{'alpha': 30},-5.772469,-12.239366,-16.651201,-10.132785,...,-12.636634,4.532799,5,-8.526545,-7.160795,-8.756514,-10.702955,-10.734824,-9.176327,1.372613


El mejor modelo en términos de la suma de error medio al cuadrado es el que usa una alpha de regularización de 0.1. La suma del error medio al cuadrado fue 8.5318. La raíz del error medio al cuadrado (RMSE) es: 2.9209

In [129]:
np.sqrt(-1 * ridge_cv.best_score_)

2.9209307372490954

###  Modelo de Lasso

Se ajusta una regresión con regularización lasso para **penalizar por parámetros muy grandes y en su caso hacer 0 algunas caraterísticas**. Se prueban distintas **alphas 0.1, 1, 10, 20 y 30** donde a más grande el alpha más se castiga aquellos parámetros con valores altos. Entre más cercano el parámetro de alpha a cero, más parecido el resultado es a la regresión lineal.


In [119]:
## Lasso
lasso = Lasso(normalize=True)
my_cv = TimeSeriesSplit(n_splits=5).split(X)
param_grid = {'alpha': [0.1, 1, 10, 20, 30]}
lasso_cv = GridSearchCV(estimator = lasso,
                        param_grid = param_grid,
                        scoring = 'neg_mean_squared_error',
                        cv = my_cv)
lasso_cv.fit(X,y)

GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x000002430C4D9468>,
       error_score='raise',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.1, 1, 10, 20, 30]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

Los resultados de los 5 modelos se encuentran a continuación.

In [120]:
pd.DataFrame(lasso_cv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,11.195976,3.496058,0.108002,0.010099,0.1,{'alpha': 0.1},-6.978841,-13.603655,-18.732158,-11.022038,...,-13.902683,4.636086,1,-10.124005,-8.551171,-10.235259,-12.359044,-12.091605,-10.672217,1.403592
1,11.549454,3.440850,0.109800,0.004261,1,{'alpha': 1},-6.978841,-13.603655,-18.732158,-11.022038,...,-13.902683,4.636086,1,-10.124005,-8.551171,-10.235259,-12.359044,-12.091605,-10.672217,1.403592
2,11.372269,3.580016,0.108001,0.004648,10,{'alpha': 10},-6.978841,-13.603655,-18.732158,-11.022038,...,-13.902683,4.636086,1,-10.124005,-8.551171,-10.235259,-12.359044,-12.091605,-10.672217,1.403592
3,11.361865,3.314325,0.104601,0.002577,20,{'alpha': 20},-6.978841,-13.603655,-18.732158,-11.022038,...,-13.902683,4.636086,1,-10.124005,-8.551171,-10.235259,-12.359044,-12.091605,-10.672217,1.403592
4,11.233864,3.187917,0.106201,0.003868,30,{'alpha': 30},-6.978841,-13.603655,-18.732158,-11.022038,...,-13.902683,4.636086,1,-10.124005,-8.551171,-10.235259,-12.359044,-12.091605,-10.672217,1.403592


El mejor modelo resultó ser el que utiliza una alpha de regularización 0.1 que obtuvó el menor MSE de 13.9026. De esta manera, el RMSE promedio de la validación cruzada del mejor modelo regresión lasso fue de:

In [130]:
np.sqrt(-1 * lasso_cv.best_score_)

3.7286301849512595

###  Extreme Gradient Boosting Machine (XGBoost) model
Se ajusta un XGBOOST para un problema de regresión.

In [32]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

Parámetros:

* **Max depth**: desde árboles cortos y profundos
* **n_esimators**: número de árboles a crear o boosting rounds
* **learning rate**: velocidad de ajuste utilizando bayes learners adicionales
* **subsample**: fracción del training set que puede ser utilizada para una boosting round. Valores chicos underfitting, valores grandes over fitting (entre 0  y 1)
* **col samples**: no se uso porque daba malos resultados
* **min child weight**:

In [44]:
# Parámetros
params_grid = {
    'max_depth' : [3,10],
    'n_estimators' : [5, 10],  #, 10, 25, 50
    'learning_rate' : [0.3], # , 0.03, 0.003, 0.0003, 0.00003
    'subsample':[1],
    'min_child_weight':[0.5]
}

params_fixed = {'objective':'reg:linear'}


In [45]:
# Validación
my_cv = TimeSeriesSplit(n_splits=5).split(X)

In [46]:
# Configuración
# Pérdiad la negative mean squared root, semilla
bst_grid = GridSearchCV(
    estimator = XGBRegressor(**params_fixed, seed = 25021987),
                                param_grid = params_grid,
                                cv = my_cv,
                                n_jobs = -1,
                                scoring = 'neg_mean_squared_error'
)

In [47]:
list(XGBRegressor().get_params().keys())

['base_score',
 'booster',
 'colsample_bylevel',
 'colsample_bytree',
 'gamma',
 'learning_rate',
 'max_delta_step',
 'max_depth',
 'min_child_weight',
 'missing',
 'n_estimators',
 'n_jobs',
 'nthread',
 'objective',
 'random_state',
 'reg_alpha',
 'reg_lambda',
 'scale_pos_weight',
 'seed',
 'silent',
 'subsample']

In [48]:
bst_grid.fit(X,y)

[01:54:09] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'


GridSearchCV(cv=<generator object TimeSeriesSplit.split at 0x000001F1AC0295C8>,
       error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=25021987,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3, 10], 'n_estimators': [5, 10], 'learning_rate': [0.3], 'subsample': [1], 'min_child_weight': [0.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [49]:
pd.DataFrame(bst_grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_min_child_weight,param_n_estimators,param_subsample,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,60.945517,29.183403,2.738785,0.621296,0.3,3,0.5,5,1,"{'learning_rate': 0.3, 'max_depth': 3, 'min_ch...",...,-8.636734,3.977369,2,-3.819979,-2.951654,-4.137040,-5.013986,-5.657646,-4.316061,0.940925
1,103.373174,52.178992,2.078028,1.093748,0.3,3,0.5,10,1,"{'learning_rate': 0.3, 'max_depth': 3, 'min_ch...",...,-8.218937,3.962476,1,-3.260720,-2.499618,-3.660976,-4.346696,-4.932570,-3.740116,0.844370
2,163.753444,75.762594,0.955608,0.120729,0.3,10,0.5,5,1,"{'learning_rate': 0.3, 'max_depth': 10, 'min_c...",...,-9.011429,4.069385,3,-2.552818,-2.053900,-3.004359,-3.213227,-3.844762,-2.933813,0.605356
3,265.178625,121.595908,1.131193,0.084365,0.3,10,0.5,10,1,"{'learning_rate': 0.3, 'max_depth': 10, 'min_c...",...,-9.071174,3.595683,4,-1.689430,-1.584405,-2.433892,-2.391216,-3.176760,-2.255141,0.577887


In [50]:
bst_grid.grid_scores_

[mean: -8.63673, std: 3.97737, params: {'learning_rate': 0.3, 'max_depth': 3, 'min_child_weight': 0.5, 'n_estimators': 5, 'subsample': 1},
 mean: -8.21894, std: 3.96248, params: {'learning_rate': 0.3, 'max_depth': 3, 'min_child_weight': 0.5, 'n_estimators': 10, 'subsample': 1},
 mean: -9.01143, std: 4.06939, params: {'learning_rate': 0.3, 'max_depth': 10, 'min_child_weight': 0.5, 'n_estimators': 5, 'subsample': 1},
 mean: -9.07117, std: 3.59568, params: {'learning_rate': 0.3, 'max_depth': 10, 'min_child_weight': 0.5, 'n_estimators': 10, 'subsample': 1}]

#### Resultados del mejor modelo xgboost

De los cuatro modelos para xgboost ajustados, a continuación se presentan los resultados del mejor, en términos de su ajuste al RMSE.

In [52]:
print("Promedio CV MSE: {}".format(bst_grid.best_score_))
print("Mejores parámetros")
for key, value in bst_grid.best_params_.items():
    print("\t{}:: {}".format(key, value))

Promedio CV MSE: -8.21893725103605
Mejores parámetros
	learning_rate:: 0.3
	max_depth:: 3
	min_child_weight:: 0.5
	n_estimators:: 10
	subsample:: 1


In [ ]:
El mejor modelo obtuvó un RMSE de:

In [54]:
np.sqrt(8.21)

2.8653097563788807

## Resultados del modelado

A manera de resumen, se estimaron **14 modelos**. Se intentaron otras combinaciones de xgboost que por motivos de memoria no pudieron adjuntarse a la tabla anterior pero ninguno superó estos resultados. 

El mejor Ridge:
* 2.92

El mejor Lasso:
* 3.72

El mejor xgboost:
* 2.86

En la siguiente sección se procederá a ajustar este modelo de xgboost sobre todo el conjunto de entrenamiento.

## Construcción del conjunto de prueba

A continuación se construye la base de datos de prueba que servirá para predecir. Las únicas variables proporcionadas por la competencia fueron:

* shop
* item


In [201]:
# Leer conjunto de prueba
test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


Ninguna de estas es la variable dependiente **item_cnt_day**.

Para correr el modelo se necesitan 44 variables de entrada en el orden que se presenta a continuación:

*Información del producto, tienda y mes de interés*

1. **shop_id**: Id de la tienda
2. **item_id**: Id del producto
3. **date_block_num**: Id del mes, en este caso 33 para el mes de octubre de 2015
4. **item_cnt_day**: Ventas promedio mensuales para el artículo
5. **item_category_id**: Id de la categoría del producto

*Rezago mensual: datos agrupados por producto, tienda y categoría*

6. **item_id_avg_item_price_lag_1**: artículo de interés en todas las tiendas, precio promedio del mes anterior
7. **item_id_sum_item_cnt_day_lag_1**: artículo de interés en todas las tiendas, ventas totales del mes anterior
8. **item_id_avg_item_cnt_day_lag_1**: artículo de interés en todas las tiendas, ventas promedio del mes anterior
9. **shop_id_avg_item_price_lag_1**: en la tienda de interés, precio promedio del producto en el mes anterior.
10. **shop_id_sum_item_cnt_day_lag_1**: en la tienda de interés, ventas totales del producto en el mes anterior.
11. **shop_id_avg_item_cnt_day_lag_1**: en la tienda de interés, ventas promedio del producto en el mes anterior.
12. **item_category_id_avg_item_price_lag_1**: en la categoría del producto de interés, precio promedio en el mes anterior.
13. **item_category_id_sum_item_cnt_day_lag_1**: en la categoría del producto de interés, ventas totales en mes anterior.
14. **item_category_id_avg_item_cnt_day_lag_1**: en la categoría del producto de interés, ventas promedio del mes anterior.
15. **item_cnt_day_lag_1**: artículo y tienda de interés, ventas en el mes anterior en la tienda de in 

*Rezago bimestral: datos agrupados por producto, tienda y categoría*

16. **item_id_avg_item_price_lag_2**: ídem, dos meses atrás
17. **item_id_sum_item_cnt_day_lag_2**: ídem, dos meses atrás
18. **item_id_avg_item_cnt_day_lag_2**: ídem, dos meses atrás
19. **shop_id_avg_item_price_lag_2**: ídem, dos meses atrás
20. **shop_id_sum_item_cnt_day_lag_2**: ídem, dos meses atrás
21. **shop_id_avg_item_cnt_day_lag_2**: ídem, dos meses atrás
22. **item_category_id_avg_item_price_lag_2**: ídem, dos meses atrás
23. **item_category_id_sum_item_cnt_day_lag_2**: ídem, dos meses atrás
24. **item_category_id_avg_item_cnt_day_lag_2**: ídem, dos meses atrás
25. **item_cnt_day_lag_2**:

*Rezago cuatrimestral: datos agrupados por producto, tienda y categoría*

26. **item_id_avg_item_price_lag_3**: ídem, tres meses atrás
27. **item_id_sum_item_cnt_day_lag_3**: ídem, tres meses atrás
28. **item_id_avg_item_cnt_day_lag_3**: ídem, tres meses atrás
29. **shop_id_avg_item_price_lag_3**: ídem, tres meses atrás
30. **shop_id_sum_item_cnt_day_lag_3**: ídem, tres meses atrás
31. **shop_id_avg_item_cnt_day_lag_3**: ídem, tres meses atrás
32. **item_category_id_avg_item_price_lag_3**: ídem, tres meses atrás
33. **item_category_id_sum_item_cnt_day_lag_3**: ídem, tres meses atrás
34. **item_category_id_avg_item_cnt_day_lag_3**: ídem, tres meses atrás
35. item_cnt_day_lag_3**: ídem, tres meses atrás

*Rezago semestral: datos agrupados por producto, tienda y categoría*

36. **item_id_avg_item_price_lag_6**: ídem, seis meses atrás
37. **item_id_sum_item_cnt_day_lag_6**: ídem, seis meses atrás
38. **item_id_avg_item_cnt_day_lag_6**: ídem, seis meses atrás
39. **shop_id_avg_item_price_lag_6**: ídem, seis meses atrás
40. **shop_id_sum_item_cnt_day_lag_6**: ídem, seis meses atrás
41. **shop_id_avg_item_cnt_day_lag_6**: ídem, seis meses atrás
42. **item_category_id_avg_item_price_lag_6**: ídem, seis meses atrás
43. **item_category_id_sum_item_cnt_day_lag_6**: ídem, seis meses atrás
44. **item_category_id_avg_item_cnt_day_lag_6**: ídem, seis meses atrás
45. **item_cnt_day_lag_6**: ídem, tres meses atrás

Es necesario preprocesar el conjunto de prueba de manera similar al de entrenamiento. Por lo que es necesario agregar y utilizar las siguientes variables:

*  date_block_num = 34 (noviembre de 2015)
*  category_id
*  lagging


In [202]:
# Mes correspondiente a noviembre de 2015.
test['date_block_num'] = 34

In [203]:
# Pegamos catálogos
test = pd.merge(test, items, on='item_id', how='left')

In [204]:
from tqdm import tqdm_notebook
# Pintar una barra de progreso en cada lag
for lag in tqdm_notebook(lags):
    # Crear rezagos con la indicadora del mes
    sales_new_df = mean_transactions.copy()
    sales_new_df.date_block_num += lag
    # Seleccionar columnas
    sales_new_df = sales_new_df[['date_block_num','shop_id','item_id']+lag_variables]
    # Editar nombres de columnas
    sales_new_df.columns = ['date_block_num','shop_id','item_id']+ [lag_feat+'_lag_'+str(lag) for lag_feat in lag_variables]
    # Merge entre test y sales new
    test = pd.merge(test, sales_new_df,on=['date_block_num','shop_id','item_id'] ,how='left')

In [205]:
# Verificamos que las columnas del conjunto de entrenamiento y prueba esten en el mismo orden
_test = set(test.drop(['ID', 'item_name'], axis=1).columns)
_training = set(training.drop('item_cnt_day',axis=1).columns)
for i in _test:
    assert i in _training
for i in _training:
    assert i in _test

In [206]:
assert _training == _test

In [207]:
test = test.drop(['ID', 'item_name'], axis=1)

In [208]:
for feat in test.columns:
    if 'item_cnt' in feat:
        test[feat]=test[feat].fillna(0)
    elif 'item_price' in feat:
        test[feat]=test[feat].fillna(test[feat].median())

Código para verificar que la base de prueba se generó sin errores.

In [209]:
test[['shop_id','item_id']+['item_cnt_day_lag_'+str(x) for x in [1,2,3]]].head()

,shop_id,item_id,item_cnt_day_lag_1,item_cnt_day_lag_2,item_cnt_day_lag_3
0,5,5037,0.0,1.0,3.0
1,5,5320,0.0,0.0,0.0
2,5,5233,1.0,3.0,1.0
3,5,5232,0.0,0.0,1.0
4,5,5268,0.0,0.0,0.0


In [210]:
print(training[training['shop_id'] == 5][training['item_id'] == 5037][training['date_block_num'] == 33]['item_cnt_day'])
print(training[training['shop_id'] == 5][training['item_id'] == 5037][training['date_block_num'] == 32]['item_cnt_day'])
print(training[training['shop_id'] == 5][training['item_id'] == 5037][training['date_block_num'] == 31]['item_cnt_day'])

10849431    0.0
Name: item_cnt_day, dtype: float64
10605772    1.0
Name: item_cnt_day, dtype: float64
10396094    3.0
Name: item_cnt_day, dtype: float64


In [211]:
test.to_csv("Data_Modelos/Prueba_Modelos.csv",index = False, header=True)

## Comparación de modelos

En esta sección se entrenaron tres tipos de modelos: una regresión Ridge, una regresión Lasso y un XGBOOST. El mejor modelo en términos de la raíz cuadrada del error medio cuadrado fue el XGBOOST y el peor fue la regresión lineal. Siendo que el primero es más flexible y el segundo el más rígido. El desempeño de  Lasso y Ridge estuvo entre estos dos modelos, lo que indica que la regularización de los parámetros es adecuada.

En la siguiente sección de Evaluación de Modelos se utilizará el modelo XGBOOST para ajustarse sobre todo el conjunto de entrenamiento. Posteriormente, se ajustará dicho modelo en el conjunto de prueba para predecir las ventas por artículo, tienda y mes.